In [ ]:
import sys

orig_stdout = sys.stdout
f = open('out.txt', 'w')
sys.stdout = f

sys.stdout = orig_stdout

In [2]:
import numpy as np
import pandas as pd
from collections import defaultdict

In [ ]:
def combine_dict(list_of_dictionaries):
    dic = defaultdict(lambda: list())
    for d in list_of_dictionaries:  
        for item in d.items():
            dic[item[0]].append(item[1])
    return dic
    
def scale(df,col,on_range = True):
    if on_range: # Scaling based on Range
        df[col] -= df[col].min()
        df[col] /= df[col].max()
    else: # Scaling based on SD
        sd = df[col].std()
        df[col] -= df[col].mean()
        df[col] /= sd
    return

In [3]:
fil = pd.read_excel('./Course_Advisory_Data.xlsx')
pre_req = pd.read_excel('./Prerequisites of All Courses.xlsx')
pre_req.set_index('Code',inplace=True)
pre_req.drop(['Sr#','Type','Status','Equivalence'],axis=1,inplace=True)
fil.dropna(inplace= True)

In [ ]:
grades = [ 'I', 'W', 'FA', 'L1', 'LL', 'L2', 'F/R', 'CN']

rep = -1

for g in grades:
    fil.loc[fil['Grade'] == g, 'Grade Point'] = rep
    rep -= 1

# help(fil.where)
# fil['FirstName'][df.ID == 103] = "Matt"

# x['Grade Point'].unique()

In [4]:
# Dictionary for Course Code and Course Title and Credit Hours
# course_guide = combine_dict([dict(zip(fil['Course Code'], fil['Course Title'])),dict(zip(fil['Course Code'], fil['Credit Hours']))])

course_guide = dict(zip(fil['Course Title'], fil['Credit Hours']))
# crs = pre_req.index.to_list()
# for c in crs:
#     if c not in course_guide:
#         course_guide[c] = pre_req.ix[c].to_list()[0:2]

# Dictionary for Grade and Grade Point
# grade_dictionary = dict(zip(fil['Grade'], fil['Grade Point']))
grade_dictionary = dict(zip(fil['Grade Point'], fil['Grade']))

In [ ]:

file = ['Advanced Research Methods',
        'Research in Marketing',
        'MS Thesis - I',
        'MS Thesis - II',
        'PhD Thesis - I',
        'PhD Thesis - II',
        'PhD Thesis - III',
        'PhD Thesis - IV',
        'Applied Programming', 
        'Research Methodology'
        ]

sem =   [
    20111,
    20121,
    20123,
    20131
    ]

for line in file:
    indexNames = fil[ fil['Course Title'] == line ].index
    fil.drop(indexNames,inplace=True)
for s in sem:
    indexNames = fil[ fil['Semester'] == s ].index
    fil.drop(indexNames,inplace=True)

fil.dropna(inplace= True)

In [ ]:
cols = fil.columns.to_list()
semesters = list(fil.Semester.unique())
# roll_nos = list(fil['Sr. No'].unique())

In [ ]:
semesters = [
    'Fall 2016',
    'Spring 2017',
    'Summer 2017',
    'Fall 2017',
    'Spring 2018',
    'Summer 2018',
    'Fall 2018',
    'Spring 2019',
    'Summer 2019',
    'Fall 2019'
        ]

In [ ]:
data = fil
data.set_index('Sr. No',inplace= True)
data = data.drop(labels = ['Grade','Credit Hours'],axis = 1)

In [ ]:
roll_nos = data.index.unique().to_list()
# scale(data, 'CGPA')
# scale(data, 'SGPA')
# scale(data, 'Grade Point')

modified_data = pd.DataFrame(columns=['Roll_No'],data=roll_nos)
modified_data.set_index(['Roll_No'],inplace=True)

cols = ['Course Title','Grade_Point','Semester','Warning','SGPA','CGPA']

for c in cols:
    modified_data[c] = np.empty((len(roll_nos), 0)).tolist()

In [ ]:
roll_nos.sort()
for roll in roll_nos:
    for semester in semesters:
        semester = str(semester)
        try:
            df = pd.DataFrame(data.ix[roll])
            df = df.groupby(['Semester']).get_group(semester)
            modified_data.ix[roll].Semester.append([semester])
            modified_data.ix[roll]['Course Title'].append(list(df['Course Title'].to_list()))
            modified_data.ix[roll].Grade_Point.append(list(df['Grade Point'].to_list()))
            modified_data.ix[roll].Warning.append(list([df['Warning'].to_list()[0]]))
            modified_data.ix[roll].SGPA.append(list([df['SGPA'].to_list()[0]]))
            modified_data.ix[roll].CGPA.append(list([df['CGPA'].to_list()[0]]))
        except KeyError:
            pass

In [ ]:
# modified_data.to_excel('New_Data.xlsx')
modified_data.to_pickle('New_Data_pickle.xlsx')

In [ ]:
data = pd.read_pickle('./New_Data_pickle.xlsx')
cols = data.columns.to_list()
students = data.index.to_list()

In [ ]:

xyData = pd.DataFrame(columns=['Taken_Courses','Grades','CGPA','SGPA','Warning','Recommended'])

students = data.index.to_list() # Enrolled Students

for student in students:
    student_data = data.loc[student]
    for i in range(len(data.loc[student].Semester)-1):
        # print(student_data['Course Title'])
        xyData = xyData.append({'Taken_Courses':student_data['Course Title'][i],
        'Grades':student_data.Grade_Point[i],
        'CGPA': student_data.CGPA[i],
        'SGPA':student_data.SGPA[i],
        'Warning':student_data.Warning[i],
        'Recommended':student_data['Course Title'][i+1]} ,
        ignore_index=True)
        if len(student_data['Course Title'][i]) != len(student_data.Grade_Point[i]):
            print(len(student_data['Course Title'][i]),len(student_data.Grade_Point[i]))


xyData.to_pickle('xydata.xlsx')

In [7]:
xyData = pd.read_pickle('xydata.xlsx')
ratio = 0.8
size = int(len(xyData) * ratio)
x_train = xyData.values[:size,:5]
y_train = xyData.values[:size,5:]

# print(x_train.shape,y_train.shape)

x_test = xyData.values[size:,:5]
y_test = xyData.values[size:,5:]

# print(x_test.shape,y_test.shape)

In [8]:

class KNN:
	def __init__(self, k=1):
		self.K = k

	def train(self, x, y):
		self.X = x
		self.Y = y

	def predict(self, x_pred):

		scores = []
		for x in self.X:
			# ['Taken_Courses', 'Grades', 'CGPA', 'SGPA', 'Warning', 'Recommended']
			if len(set(x[0])&set(x_pred[0])) != 0:
				score = 0
				uncommon = len(set(x[0])) + len(set(x_pred[0])) - len(set(x[0])&set(x_pred[0]))*2
				common = list(set(x[0])&set(x_pred[0]))
				
				for c in common:
					id1 = x_pred[0].index(c)
					id2 = x[0].index(c)
					score += (x_pred[1][id1] - x[1][id2])**2
				score **=0.5
				score += ((x_pred[2][0] - x[2][0])**2)
				score += ((x_pred[3][0] - x[3][0])**2)
				score += ((x_pred[4][0] - x[4][0])**2)
				score += uncommon
				scores.append(score)
			else:
				scores.append(999)
			
		scores = np.array(scores)	
		min_vals = np.argpartition(scores,self.K)[:self.K]
		l = [j for sub in list(self.Y[:, 0][min_vals]) for j in sub]
		d = dict((x, l.count(x)) for x in set(l))
		d = {k: v for k, v in sorted(d.items(),reverse= True, key=lambda item: item[1])}
		return d
	def evaluate(self):
		pass

r = 400
classifier = KNN(7)
classifier.train(x_train,y_train)
classifier.predict(x_test[r])

{'Statistical Inference': 7,
 'Statistical Inference - Lab': 7,
 'Corporate Accounting-I': 7,
 'Business Communication - I': 7,
 'IT in Business': 7,
 'Fundamentals of Management': 7,
 'IT in Business - Lab': 7,
 'Electrical Network Analysis': 1,
 'Electronic Devices and Circuits': 1,
 'Complex Variables and Transforms ': 1,
 'Data Structures and Algorithms - Lab': 1,
 'Electrical Network Analysis Lab': 1,
 'Data Structures and Algorithms': 1,
 'Electronic Devices and Circuits - Lab': 1,
 'Linear Algebra': 1}

In [57]:
def adviseMe(classifier, semesterData, course_guide):
    courses = classifier.predict(semesterData)
    creditLimit = 19
    if semesterData[4][0] != 0: # Check Warning Count
        creditLimit = 15
    assignedLimit = 0
    coursesList = list(courses.keys())
    suggestedCourses = []
    otherSuggestions = []
    for c in coursesList:
        if assignedLimit + course_guide.get(c) <= creditLimit:
            suggestedCourses.append(c)
            assignedLimit += course_guide.get(c)
        elif assignedLimit >= creditLimit:
            otherSuggestions.append(c)
    return suggestedCourses,assignedLimit,otherSuggestions

k = 2
r = 1000
classifier = KNN(k)
classifier.train(x_train,y_train)

cour, lim, other = adviseMe(classifier,x_test[r], course_guide)

for idx,y in enumerate(x_test[r][0]):
    print(y,grade_dictionary.get(x_test[r][1][idx]))

print('\nWarning Count : ',x_test[r][4][0])
print('\n\n')

print(*cour, '\nCredit Hours : '+str(lim), sep= '\n')

print('\n\n\n*********Other Possible Suggestions****************\n' ,*other, sep= '\n')

Object Oriented Programming - Lab B-
Pakistan Studies B+
Communication & Presentation Skills C+
Digital Logic Design C
Communication & Presentation Skills - Lab A
Differential Equations (Cal II) B
Digital Logic Design - Lab A
Object Oriented Programming A-

Warning Count :  0



Data Structures
Digital Logic Design - Lab
Data Structures - Lab
Digital Logic Design
Discrete Structures
Computer Architecture
Probability and Statistics

Credit Hours : 18



*********Other Possible Suggestions****************



In [ ]:
k = 7
r = 400
classifier = KNN(k)
classifier.train(x_train,y_train)

print(adviseMe(classifier,x_test[r],course_guide))



In [ ]:
print(course_guide.get('Data Structures'))

In [ ]:
# x,yy = classifier.predict(x_test[r])

# print('\n\n',set(x_test[r][0])&set(x[0]))

# print("\n\nGiven\n\n")
# for idx,y in enumerate(x_test[r][0]):
#     print(y,grade_dictionary.get(x_test[r][1][idx]))

# print("\n\nMatched\n\n")
# for idx,y in enumerate(x[0]):
#     print(y,grade_dictionary.get(x[1][idx]))


# print('\n\n',set(y_test[r][0])&set(yy))

# print("\n\nLabels\n\n")
# for idx,y in enumerate(y_test[r][0]):
#     print(y)

# print("\n\nPredicted\n\n")
# for idx,i in enumerate(yy):
#     print(i)

grade_dictionary